In [3]:
require('nn')
require('hdf5')

f = hdf5.open("data_glove_5.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
Y_valid = f:read("valid_output"):all()

X_test = f:read("test_input"):all()

X_train = torch.cat(X_train, X_valid, 1)
Y_train = torch.cat(Y_train, Y_valid, 1)

wiki = (f:read("wiki"):all()):sub(2,-1)
embedded_size = wiki:size(2)
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]

f:close()

window_size = X_train:size(2)/2
X_train_word = X_train:sub(1,X_train:size(1),1,window_size)
--X_valid_word = X_valid:sub(1,X_valid:size(1),1,window_size)
X_train_cap = X_train:sub(1,X_train:size(1),window_size+1,2*window_size)
--X_valid_cap = X_valid:sub(1,X_valid:size(1),window_size+1,2*window_size)
X_test_word = X_test:sub(1,X_test:size(1),1,window_size)

X = X_train_word:clone()
--X_v = X_valid_word:clone()
X_t = X_test_word:clone()

In [4]:
X_test_cap = X_test:sub(1,X_test:size(1),window_size+1,2*window_size)

In [8]:
X_test_cap = X_test:sub(1,X_test:size(1),window_size+1,2*window_size)

function score(input_word,input_cap, output)
    local pred = torch.zeros(input_word:size(1))
    local score = torch.zeros(input_word:size(1))
    
    f = io.open("NN_7.csv", "w")
    f:write("ID,Class\n")
    
    for i = 1,pred:size(1) do
        a = input_word[i]:view(1,window_size)
        b = input_cap[i]:view(1,window_size)
        vec = h:forward{a:double(),b:double()}
        
        local max, ind = (vec):max(2)

        pred[i] = ind
        
        f:write(tostring(i) .. "," .. tostring(pred[i]) .. "\n")
        
        if output[i] == pred[i] then
            score[i] = 1
        else
            score[i] = 0
        end
    end
    f:close()
    return torch.mean(score)
end

In [6]:
X_ = X
y = Y_train
batch = 1000
D_h = 500

lookup1 = nn.LookupTable(wiki:size(1),wiki:size(2))
lookup1.weight = wiki
linear1 = nn.Linear(window_size*embedded_size + window_size, D_h)
tanh = nn.HardTanh()
linear2 = nn.Linear(D_h, nclasses)
h = nn.Sequential()
h1 = nn.Sequential()
h1:add(lookup1)
h1:add(nn.Reshape(window_size*embedded_size, True))
hw = nn.Linear(5,5)--nn.SelectTable(-1)
mlp = nn.ParallelTable()
mlp:add(h1)
mlp:add(hw)
h:add(mlp)
h:add(nn.JoinTable(2))
h:add(linear1)
h:add(tanh)
h:add(linear2)

mse = nn.CrossEntropyCriterion()

In [9]:
batch = 1000
eta = .1
nEpochs = 50000
x = torch.Tensor(1,5)
cap = torch.Tensor(1,5)

for i = 1, nEpochs do
    
    
    bsize= batch
    local idx = torch.randperm(X_:size(1)):sub(1,bsize)

    x = torch.Tensor(bsize, X_:size(2))
    cap = torch.Tensor(bsize, X_train_cap:size(2))
    
    y = y:reshape(y:size(1), 1)
    
    y_ = torch.Tensor(bsize, 1)

    for i=1,bsize do
        cap[i] =X_train_cap[idx[i]]
        x[i] = X_[idx[i]]
        y_[i] = y[idx[i]]
    end
    
    y_ = y_:squeeze()
    --concat
    -- zero out our gradients
    h:zeroGradParameters()
    -- do forward pass
    preds = h:forward{x,cap}
    -- get loss
    loss = mse:forward(preds, y_)
    if loss == 0 then break end    
    -- backprop
    lookup1:zeroGradParameters()
    dLdpreds = mse:backward(preds, y_) -- gradients of loss wrt preds
    dLdparams = h:backward({x,cap}, dLdpreds)
    h:updateParameters(eta)
    if math.fmod(i,100) == 0 then
        
        print("epoch " .. i .. ", loss: " .. loss)
    end
end

score(X_t, X_test_cap, Y_valid)

epoch 100, loss: 0.057236149692612	


epoch 200, loss: 0.044597792385355	


epoch 300, loss: 0.040292140266806	


epoch 400, loss: 0.073164681121231	


epoch 500, loss: 0.063163209855438	


...

epoch 49600, loss: 0.056993647369455	


epoch 49700, loss: 0.029876021994953	


epoch 49800, loss: 0.039226786135942	


epoch 49900, loss: 0.065630744620304	


epoch 50000, loss: 0.037886479658498	


In [16]:
score(X_v:sub(1,1000), X_valid_cap:sub(1,1000), Y_valid:sub(1,1000))

0.955	


0.068020602023193	


In [41]:
score(X_v, X_valid_cap, Y_valid)

0.96362891478514	
